In [23]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import load_model

from settings import *
import eb_models
from preprocessing import load_npz, make_dataset, normalize

In [6]:
cut_len = 1024

In [13]:
h5list = glob.glob(os.path.join(modeldir, "*.h5"))

In [8]:
data0_list = []
data1_list = []
for sector in [1, 2, 3, 5]:
    npzpath0 = os.path.join(datdir, "%s_0.npz" % sector)
    npzpath1 = os.path.join(datdir, "%s_1.npz" % sector)
    data1 = load_npz(npzpath1)
    data0 = load_npz(npzpath0)[:data1.shape[0]]
    data0 = make_dataset(data0, cut_len)
    data1 = make_dataset(data1, cut_len)
    data0_list.append(data0)
    data1_list.append(data1)
data0 = np.vstack(tuple(data0_list))
data1 = np.vstack(tuple(data1_list))
label0 = np.zeros(data0.shape[0])
label1 = np.ones(data1.shape[0])
data = np.vstack((data0, data1))
label = np.hstack((label0, label1))
data = normalize(data)

In [26]:
# , custom_objects={"class_name": class_name}
for h5path in h5list:
    model = load_model(h5path, compile=True)

KeyError: 0